<a href="https://colab.research.google.com/github/carolvieirav/Case-Hurb/blob/main/An%C3%A1lisePM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
df1=pd.read_excel('/content/Base Case Paid Marketing.xlsx', sheet_name=0)
df2=pd.read_excel('/content/Base Case Paid Marketing.xlsx', sheet_name=1)

In [ ]:
df1.head()

,Data,Destino,Buscas,Clicks,Custo
0,2019-01-01,Abadiania,13,0,0.000000
1,2019-01-01,Abaete,3,4,0.533333
2,2019-01-01,Abelardo Luz,22,1,0.130000
3,2019-01-01,Aberdeenshire,3,0,0.000000
4,2019-01-01,Abrantes,7,0,0.000000


In [ ]:
df2.head()

,Cidade,Data,Pedidos,Faturamento
0,Alto Caparao,2019-01-01,3,1204.245000
1,Chapada dos Guimarães,2019-01-01,1,4041.160000
2,Jericoacoara,2019-01-01,4,21316.266667
3,Punta Del Este,2019-01-01,1,554.260000
4,Marilia,2019-01-01,1,670.300000
